#Import BERT model and freeze layers

In [1]:
# A dependency of the preprocessing for BERT inputs
!pip install -U "tensorflow-text==2.13.*"

!pip install "tf-models-official==2.13.*"

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9ee69e7104f37cd5216c544043d3d6b0f72d4fe2d29e692981a264adde8ed4d9
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_hub as hub
from transformers import BertTokenizer, TFBertModel
from keras.layers import Layer, Input, Dense
from keras.models import Model
import tensorflow_text as text
from sklearn.preprocessing import LabelEncoder


In [2]:
# Read excel using pandas
df = pd.read_excel('/content/drive/MyDrive/news_less_clean.xlsx')
df.shape
df.head()

,Date,URL,Title,Source,Country,LABEL
0,20240815T010000Z,https://borneobulletin.com.bn/explosions-repor...,Explosions reported near two ships off Yemen :...,borneobulletin.com.bn,Brunei,2
1,20240716T194500Z,https://www.hindustantimes.com/india-news/crew...,"Crew , including 13 Indians , still missing af...",hindustantimes.com,India,2
2,20240809T100000Z,https://www.yahoo.com/news/multiple-attacks-ta...,Multiple attacks target merchant ship off Yeme...,yahoo.com,United States,3
3,20240717T041500Z,https://timesofoman.com/article/147862-oil-tan...,Oil tanker with 13 Indians on board sinks off ...,timesofoman.com,Oman,2
4,20240812T201500Z,https://menafn.com/1108546043/Multiple-Attacks...,Multiple Attacks Target Merchant Ship Off Yemen,menafn.com,Qatar,3


In [3]:
# Only use headline and label
df_to_split = df[['Title', 'LABEL']]
df_to_split.head()

,Title,LABEL
0,Explosions reported near two ships off Yemen :...,2
1,"Crew , including 13 Indians , still missing af...",2
2,Multiple attacks target merchant ship off Yeme...,3
3,Oil tanker with 13 Indians on board sinks off ...,2
4,Multiple Attacks Target Merchant Ship Off Yemen,3


In [4]:
# Map labels to numerical values
label_mapping = {
    1: 'Vessel Delay',
    2: 'Vessel Accidents',
    3: 'Maritime Piracy or Terrorism risk',
    4: 'Port or Important Route Congestion',
    5: 'Port Criminal Activities',
    6: 'Cargo Damage and Loss',
    7: 'Inland Transportation Risks',
    8: 'Environmental Impact and Pollution',
    9: 'Natural Extreme Events and Extreme Weather',
    10: 'Cargo or Ship Detainment',
    11: 'Unstable Regulatory and Political Environment',
    12: 'Maritime-related but not covered by existing categories',
    13: 'Non-maritime-related'
}

# Apply the mapping to the dataset
df_to_split['Category'] = df['LABEL'].map(label_mapping)

<ipython-input-4-ff044b927543>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_split['Category'] = df['LABEL'].map(label_mapping)


In [5]:
from sklearn.model_selection import train_test_split
X = df_to_split['Title']
# y = df_to_split['LABEL']
y = df_to_split['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=424)

In [6]:
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3"

bert_preprocessor = hub.KerasLayer(preprocessor_url)
bert_model = hub.KerasLayer(encoder_url, trainable=False)


In [7]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocessor(input_text)
preprocessed_text
outputs = bert_model(preprocessed_text)

# Build custom model to extend and classify into 13 labels
# Add classification layers
x = Dense(64, activation='relu')(outputs['pooled_output'])
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
output = Dense(13, activation='softmax')(x)

model = tf.keras.Model(inputs=input_text, outputs=output)

optimzer = tf.keras.optimizers.Adam(learning_rate=0.001)
# Compile the model
model.compile(optimizer=optimzer, loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [8]:
# # # One-hot encode the target variables
# y_train = tf.keras.utils.to_categorical(y_train-1, num_classes=13)
# y_test = tf.keras.utils.to_categorical(y_test-1, num_classes=13)
# Initialize LabelEncoder
encoder = LabelEncoder()

# Convert string labels to integers
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# One-hot encode the target variables
y_train = tf.keras.utils.to_categorical(y_train, num_classes=13)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=13)

# # Train model
model.fit(X_train, y_train, epochs=7, batch_size=64)

Epoch 1/7
9/9 [==============================] - 176s 18s/step - loss: 2.3472 - accuracy: 0.2211
Epoch 2/7
9/9 [==============================] - 168s 19s/step - loss: 2.2441 - accuracy: 0.2175
Epoch 3/7
9/9 [==============================] - 169s 19s/step - loss: 2.1896 - accuracy: 0.2579
Epoch 4/7
9/9 [==============================] - 168s 19s/step - loss: 2.1535 - accuracy: 0.2614
Epoch 5/7
9/9 [==============================] - 174s 19s/step - loss: 2.1307 - accuracy: 0.2456
Epoch 6/7
9/9 [==============================] - 170s 19s/step - loss: 2.1052 - accuracy: 0.2719
Epoch 7/7
9/9 [==============================] - 169s 19s/step - loss: 2.0821 - accuracy: 0.2982


In [9]:
# Test Data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

5/5 [==============================] - 40s 8s/step - loss: 2.0726 - accuracy: 0.3217
Test Loss: 2.0725595951080322, Test Accuracy: 0.3216783106327057
